In [3]:
!pip install torch==1.4.0 torchvision==0.5.0  #need to downgrade the pytorch version

     |████████████████████████████████| 753.4MB 22kB/s 
     |████████████████████████████████| 4.0MB 46.1MB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101


In [4]:
!pip install torchtext==0.4.0

     |████████████████████████████████| 61kB 3.4MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [5]:
!pip install OpenNMT-py

     |████████████████████████████████| 194kB 4.5MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 2.2MB 7.7MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.2.3-cp36-none-any.whl size=19328 sha256=12e665eda7167661461896fb5c6938619ff9eceee98912287c007cd2a862785f
  Stored in directory: /root/.cache/pip/wheels/bd/d6/53/034032da9498bda2385cd50a51a289e88090b5da2d592b1fdf
Successfully built configargparse
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.30.0 which is incompatible.
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [6]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 16112, done.
remote: Total 16112 (delta 0), reused 0 (delta 0), pack-reused 16112
Receiving objects: 100% (16112/16112), 146.81 MiB | 35.46 MiB/s, done.
Resolving deltas: 100% (11652/11652), done.


In [1]:
mkdir data model #the data for source and target should be uploaded in the data dir

**The data files should be uploaded in the data directory, my root directory is /content**

In [7]:
!python /content/OpenNMT-py/preprocess.py \
      -train_src /content/data/src-gr20-train.txt \
      -train_tgt /content/data/tgt-gr20-train.txt \
      -valid_src /content/data/src-gr20-val.txt \
      -valid_tgt /content/data/tgt-gr20-val.txt \
      -save_data /content/data/datum

[2020-08-13 12:55:51,176 INFO] Extracting features...
[2020-08-13 12:55:51,176 INFO]  * number of source features: 0.
[2020-08-13 12:55:51,176 INFO]  * number of target features: 0.
[2020-08-13 12:55:51,176 INFO] Building `Fields` object...
[2020-08-13 12:55:51,176 INFO] Building & saving training data...
[2020-08-13 12:55:51,587 INFO] Building shard 0.
[2020-08-13 12:55:58,448 INFO]  * saving 0th train data shard to /content/data/datum.train.0.pt.
[2020-08-13 12:56:05,560 INFO]  * tgt vocab size: 50004.
[2020-08-13 12:56:05,876 INFO]  * src vocab size: 50002.
[2020-08-13 12:56:06,395 INFO] Building & saving validation data...
[2020-08-13 12:56:06,826 INFO] Building shard 0.
[2020-08-13 12:56:07,182 INFO]  * saving 0th valid data shard to /content/data/datum.valid.0.pt.


In [8]:
!python /content/OpenNMT-py/train.py -gpu_ranks 0 \
      -batch_size 64 \
      -rnn_type LSTM \
      -rnn_size 200 \
      -data /content/data/datum \
      -save_model /content/model/model_gr20 

[2020-08-13 12:56:13,122 INFO]  * src vocab size = 50002
[2020-08-13 12:56:13,122 INFO]  * tgt vocab size = 50004
[2020-08-13 12:56:13,122 INFO] Building model...
[2020-08-13 12:56:17,840 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 200, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50004, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False)
      (layers): ModuleList(
        (0): LSTMCell(700, 200)
        (1): LSTMCell(200, 200)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=200, out_features=200, bias=False)
      

In [ ]:
!python /content/OpenNMT-py/translate.py \
  -model /content/model/model_gr20_step_100000.pt\ #this should be the best model in the model dir
  -src /content/data/src-gr20-test.txt \
  -output /content/data/predict.txt \  #this is the file I need to evaluate
  -replace_unk -verbose

In [ ]:
!perl /content/OpenNMT-py/tools/multi-bleu.perl /content/data/tgt-gr20-test.txt < /content/data/predict.txt